In [1]:
import pandas as pd
import numpy as np

# data = np.load('distorted_data/concatenated_data.npz')
data = np.load('datasets/training.npz')
print(len(data['original']), list(data.keys()))
data

# # load the data from every file in the distorted_data directory

# import os
# data_dir = 'distorted_data'

# data = []
# for filename in os.listdir(data_dir):
#     if filename.endswith('.npz'):
#         file_path = os.path.join(data_dir, filename)
#         loaded_data = np.load(file_path)
#         data.append(loaded_data)

# # concatenate the data from all files
# all_data = {}
# for key in data[0].keys():
#     all_data[key] = np.concatenate([d[key] for d in data], axis=0)

# # check the shape of the concatenated data
# for key, value in all_data.items():
#     print(f"{key}: {value.shape}")

# # save the concatenated data to a new npz file
# output_file = 'distorted_data/concatenated_data.npz'
# np.savez(output_file, **all_data)

34561 ['original', 'label', 'Uniform_Noise', 'Rotate_90deg']


In [2]:
import os

def import_data(directory, save_path=None, save=False):

    data = []
    for filename in os.listdir(directory):
        if filename.endswith('.npz'):
            file_path = os.path.join(directory, filename)
            loaded_data = np.load(file_path)
            data.append(loaded_data)

    # concatenate the data from all files
    all_data = {}
    for key in data[0].keys():
        all_data[key] = np.concatenate([d[key] for d in data], axis=0)

    # check the shape of the concatenated data
    for key, value in all_data.items():
        print(f"{key}: {value.shape}")  

    if save:
        if save_path is None:
            save_path = f'datasets/{directory}_concatenated_data.npz'
        np.savez(save_path, **all_data)
        print(f"Data saved to {save_path}")

    return all_data

In [3]:
train_data = import_data('imported_distorted/train', save_path='datasets/train.npz', save=True)
val_data =  import_data('imported_distorted/val', save_path='datasets/validation.npz', save=True)
test_data = import_data('imported_distorted/test', save_path='datasets/test.npz', save=True)

original: (34561, 224, 224)
label: (34561, 1)
Uniform_Noise: (34561, 224, 224)
Rotate_90deg: (34561, 224, 224)
Data saved to datasets/train.npz
original: (6491, 224, 224)
label: (6491, 1)
Uniform_Noise: (6491, 224, 224)
Rotate_90deg: (6491, 224, 224)
Data saved to datasets/validation.npz
original: (17778, 224, 224)
label: (17778, 1)
Uniform_Noise: (17778, 224, 224)
Rotate_90deg: (17778, 224, 224)
Data saved to datasets/test.npz


In [4]:
test_data.keys()

dict_keys(['original', 'label', 'Uniform_Noise', 'Rotate_90deg'])

In [5]:
# keys = list(data.keys())

# images = [data[keys[0]], data[keys[2]], data[keys[3]]]
# labels = data[keys[1]]

In [6]:
# images[0][0]

In [7]:
def normalize_image(image, mean=0.5, std=0.5):
    """
    Normalize an image tensor to have a mean and standard deviation.
    """
    return (image - mean) / std

def normalize_images(images, mean=0.5, std=0.5):
    """
    Normalize a list of images.
    """
    return [normalize_image(image, mean, std) for image in images]

In [8]:
# normalized_images = []
# for image in images:
#     normalized_images.append(normalize_images(image))

In [9]:
def plot_sample_images(images, labels, n=3):
    """
    Plot a sample of images with their corresponding labels.
    """
    import matplotlib.pyplot as plt
    import numpy as np

    plt.figure(figsize=(15, 5))
    for i in range(n):
        plt.subplot(1, n, i + 1)
        plt.imshow(images[i][0], cmap='gray')
        plt.title(f'Label: {labels[i]}')
        plt.axis('off')
    plt.show()

# plot_sample_images(images, labels)
# plot_sample_images(normalized_images, labels)

In [10]:
# images[0][0]

In [11]:
# normalized_images[0][0]

In [12]:
# type(normalized_images[0][0])
# normalized_images[0][0].shape

In [13]:
# zero_labels = np.zeros_like(labels)
# one_labels = np.ones_like(labels)

# print(zero_labels.shape, one_labels.shape)

# domain_labels = np.concatenate((zero_labels, one_labels, one_labels), axis=0)
# print(domain_labels.shape)

# expanded_labels = np.concatenate((labels, labels, labels), axis=0)
# print(expanded_labels.shape)

In [14]:
# # concatenate the matrices in the images list
# concatenated_images = np.concatenate(normalized_images, axis=0)
# concatenated_images.shape

In [15]:
from branched_resnet import CustomImageDataset

def preprocess_data(data, distortions = [3]):

    keys = list(data.keys())

    images = [data[keys[0]]]

    for distortion in distortions:
        images.append(data[keys[distortion]])

    labels = data[keys[1]]

    normalized_images = []
    for image in images:
        normalized_images.append(normalize_images(image))

    zero_labels = np.zeros_like(labels)
    one_labels = np.ones_like(labels)

    domain_label_list = [zero_labels]
    expanded_label_list = [labels]

    for _ in distortions:
        domain_label_list.append(one_labels)
        expanded_label_list.append(labels)

    domain_labels = np.concatenate(domain_label_list, axis=0)
    expanded_labels = np.concatenate(expanded_label_list, axis=0)

    concatenated_images = np.concatenate(normalized_images, axis=0)

    dataset = CustomImageDataset(images=concatenated_images, labels1=expanded_labels, labels2=domain_labels)

    return dataset

In [16]:
import branched_resnet as br 
from branched_resnet import CustomImageDataset
from transformers import Trainer, TrainingArguments, PreTrainedModel, ResNetConfig

config = ResNetConfig()
model = br.ResNetForMultiLabel(config=config, num_d1_classes=11, num_d2_classes=2)

for name, param in model.named_parameters():
    print(name, param.data.mean(), param.requires_grad)
    break

#train_ds = CustomImageDataset(images=concatenated_images, labels1=expanded_labels, labels2=domain_labels)

# vars(model)

resnet.embedder.embedder.convolution.weight tensor(-0.0003) True


In [17]:
train_ds = preprocess_data(train_data, distortions=[3])
val_ds = preprocess_data(val_data, distortions=[])
#test_ds = preprocess_data(test_data, distortions=[2, 3])

# D20

Train on undistorted and rotate 90

Test on the rest

In [19]:
trainer = br.train_model(train_dataset=train_ds, eval_dataset= val_ds, model=model, output_dir= "./D20_Experiment", num_epochs=50, batch_size=32)
trainer.save_model('branched_resnet_model_D20')

C:\Users\samue\AppData\Roaming\Python\Python311\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy Branch1,Precision Branch1,Recall Branch1,F1 Branch1,Accuracy Branch2,Precision Branch2,Recall Branch2,F1 Branch2
1,0.300500,0.427585,0.841473,0.807323,0.796217,0.768883,1.000000,1.000000,1.000000,1.000000
2,0.159500,0.161450,0.935449,0.925512,0.918313,0.916370,0.977661,0.500000,0.488831,0.494352
3,0.085200,0.114083,0.958712,0.957625,0.957753,0.956594,1.000000,1.000000,1.000000,1.000000
4,0.096000,0.139692,0.945155,0.943926,0.940212,0.938615,0.997073,0.500000,0.498536,0.499267
5,0.084900,0.102845,0.960099,0.956618,0.959880,0.957270,0.983670,0.500000,0.491835,0.495884
6,0.041600,0.151134,0.959174,0.960673,0.961847,0.960604,0.991527,0.500000,0.495763,0.497873
7,0.053300,0.114455,0.968264,0.970640,0.968690,0.968348,0.953012,0.500000,0.476506,0.487970
8,0.033800,0.109304,0.974272,0.968789,0.966942,0.967132,0.957171,0.500000,0.478586,0.489059
9,0.015400,0.133631,0.973502,0.975004,0.973594,0.972971,0.991219,0.500000,0.495609,0.497795
10,0.573600,0.138906,0.966415,0.966904,0.966793,0.966056,0.989216,0.500000,0.494608,0.497289


In [ ]:
# Clear up memory
train_data = None
val_data = None
train_ds = None
val_ds = None


test_ds1 = preprocess_data(test_data, distortions=[2, 3])
test_ds2 = preprocess_data(test_data, distortions=[2])

In [23]:
baseline = preprocess_data(test_data, distortions=[])

In [ ]:
# Undistorted only
trainer.evaluate(baseline)

{'eval_loss': 0.27191662788391113,
 'eval_accuracy_branch1': 0.939250759365508,
 'eval_precision_branch1': 0.9367728474503132,
 'eval_recall_branch1': 0.9402374599770941,
 'eval_f1_branch1': 0.9379728280941767,
 'eval_accuracy_branch2': 0.9902688716391045,
 'eval_precision_branch2': 0.5,
 'eval_recall_branch2': 0.49513443581955224,
 'eval_f1_branch2': 0.49755532317779727,
 'eval_runtime': 26.3996,
 'eval_samples_per_second': 673.419,
 'eval_steps_per_second': 84.206,
 'epoch': 50.0}

In [ ]:
# All available distortions and undistorted
trainer.evaluate(test_ds1)

{'eval_loss': 0.09088194370269775,
 'eval_accuracy_branch1': 0.7893838827014662,
 'eval_precision_branch1': 0.8454324238780878,
 'eval_recall_branch1': 0.7639165445180887,
 'eval_f1_branch1': 0.7857613581009594,
 'eval_accuracy_branch2': 0.33239584505193687,
 'eval_precision_branch2': 0.3737361815037255,
 'eval_recall_branch2': 0.49686410169872874,
 'eval_f1_branch2': 0.25203278179223426,
 'eval_runtime': 70.1357,
 'eval_samples_per_second': 760.44,
 'eval_steps_per_second': 95.059,
 'epoch': 50.0}

In [ ]:
# Undistorted and unseen distortion 
trainer.evaluate(test_ds2)

{'eval_loss': 0.13632291555404663,
 'eval_accuracy_branch1': 0.7131848351895601,
 'eval_precision_branch1': 0.8080319922485907,
 'eval_recall_branch1': 0.6775662692176279,
 'eval_f1_branch1': 0.70289374335409,
 'eval_accuracy_branch2': 0.4951625604679942,
 'eval_precision_branch2': 0.2516582560000312,
 'eval_recall_branch2': 0.4951625604679941,
 'eval_f1_branch2': 0.3312257567710031,
 'eval_runtime': 51.8784,
 'eval_samples_per_second': 685.371,
 'eval_steps_per_second': 85.681,
 'epoch': 50.0}